In [35]:
#pip install transformers datasets torch


In [36]:
import pandas as pd

dataset= pd.read_csv('dataset.csv')

In [37]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [38]:
dataset = dataset.head(1000)

In [40]:
#pip install transformers

In [41]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import  Trainer, TrainingArguments

In [42]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model =  BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


C:\Users\kelvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress

In [43]:

def map_score_to_sentiment(score):
    if score in [1, 2]:
        return 0  # Negative
    elif score == 3:
        return 1  # Neutral
    else:
        return 2  # Positive

In [44]:
dataset['sentiment'] = dataset['Score'].apply(map_score_to_sentiment)

dataset = dataset[['Text', 'sentiment']]



In [45]:
dataset

,Text,sentiment
0,I have bought several of the Vitality canned d...,2
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,2
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,2
...,...,...
995,BLACK MARKET HOT SAUCE IS WONDERFUL.... My hus...,2
996,"Man what can i say, this salsa is the bomb!! i...",2
997,this sauce is so good with just about anything...,2
998,Not hot at all. Like the other low star review...,0


In [46]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42)

In [47]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [48]:

# Tokenize data
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['Text'], padding='max_length', truncation=True)
    tokenized_inputs['labels'] = examples['sentiment']
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)



Map: 100%|██████████| 800/800 [00:01<00:00, 749.59 examples/s]


Map: 100%|██████████| 200/200 [00:00<00:00, 732.04 examples/s]


In [49]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }



In [50]:
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    output_dir='./results',
)

C:\Users\kelvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [52]:
trainer.train()

 33%|███▎      | 50/150 [53:17<1:52:50, 67.70s/it]












C:\Users\kelvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
                                                  

                                         
 33%|███▎      | 50/150 [57:30<1:52:50, 67.70s/it]



{'eval_loss': 0.38149863481521606, 'eval_accuracy': 0.87, 'eval_f1': 0.8443657817109146, 'eval_precision': 0.8203428571428572, 'eval_recall': 0.87, 'eval_runtime': 253.0667, 'eval_samples_per_second': 0.79, 'eval_steps_per_second': 0.051, 'epoch': 1.0}


 67%|██████▋   | 100/150 [2:03:01<1:33:02, 111.65s/it]












C:\Users\kelvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
                                                      

                                         
 67%|██████▋   | 100/150 [2:07:04<1:33:02, 111.65s/it]



{'eval_loss': 0.38189688324928284, 'eval_accuracy': 0.875, 'eval_f1': 0.8480254154447703, 'eval_precision': 0.8235666912306558, 'eval_recall': 0.875, 'eval_runtime': 242.9416, 'eval_samples_per_second': 0.823, 'eval_steps_per_second': 0.054, 'epoch': 2.0}


100%|██████████| 150/150 [2:56:32<00:00, 58.00s/it]   












                                                   

                                         
100%|██████████| 150/150 [3:00:23<00:00, 58.00s/it]

                                                   
100%|██████████| 150/150 [3:00:23<00:00, 72.15s/it]

{'eval_loss': 0.435861736536026, 'eval_accuracy': 0.885, 'eval_f1': 0.8713770106845133, 'eval_precision': 0.8624061085972852, 'eval_recall': 0.885, 'eval_runtime': 228.0741, 'eval_samples_per_second': 0.877, 'eval_steps_per_second': 0.057, 'epoch': 3.0}
{'train_runtime': 10823.071, 'train_samples_per_second': 0.222, 'train_steps_per_second': 0.014, 'train_loss': 0.39232409159342446, 'epoch': 3.0}


TrainOutput(global_step=150, training_loss=0.39232409159342446, metrics={'train_runtime': 10823.071, 'train_samples_per_second': 0.222, 'train_steps_per_second': 0.014, 'total_flos': 631472202547200.0, 'train_loss': 0.39232409159342446, 'epoch': 3.0})

In [53]:
eval_results = trainer.evaluate()

print(f"Evaluation results: {eval_results}")

100%|██████████| 13/13 [03:50<00:00, 17.70s/it]

Evaluation results: {'eval_loss': 0.435861736536026, 'eval_accuracy': 0.885, 'eval_f1': 0.8713770106845133, 'eval_precision': 0.8624061085972852, 'eval_recall': 0.885, 'eval_runtime': 248.226, 'eval_samples_per_second': 0.806, 'eval_steps_per_second': 0.052, 'epoch': 3.0}


In [64]:
import torch

new_examples = [
    "My son loved the original recipe Chick Chick but Happy Baby has now changed the recipe.  My son retched upon the first bite of the new meal.  Why do companies try to improve upon something that already works.  I'm guessing to cut corners somehow (ie. cheaper ingredients).  It's a shame.  I now have a case of the new recipe that won't be eaten.",
    "Nothing amazing about this little blade.  Used to make Zuchinni Noodles.  Worked Perfectly.<br />No Problems yet.<br />Kind of wish I had gone with one of those multi blade peelers but this was a little cheeper, and I ofern find multi use items tend to break often.",
    "My 10 month old baby hates this stuff!!  I've tried giving it to him on 4 separate occasions-but everytime he tastes it--he makes a disgusted look and won't eat anymore.  I even tried mixing it with his favorite foods and he just takes one bite and looses his appetite and won't eat anymore of anything for the rest of the night!"
]

def tokenize_examples(examples):
    return tokenizer(examples, padding='max_length', truncation=True, return_tensors='pt')

inputs = tokenize_examples(new_examples)

# Move tensors to the same device as the model
inputs = {key: value.to(model.device) for key, value in inputs.items()}

with torch.no_grad(): 
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1)

def label_from_prediction(pred):
    labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    return [labels[prediction] for prediction in pred]

predicted_labels = label_from_prediction(predictions.cpu().numpy())

for text, label in zip(new_examples, predicted_labels):
    print(f"Text: {text}\nPredicted Sentiment: {label}\n")


Text: My son loved the original recipe Chick Chick but Happy Baby has now changed the recipe.  My son retched upon the first bite of the new meal.  Why do companies try to improve upon something that already works.  I'm guessing to cut corners somehow (ie. cheaper ingredients).  It's a shame.  I now have a case of the new recipe that won't be eaten.
Predicted Sentiment: Negative

Text: Nothing amazing about this little blade.  Used to make Zuchinni Noodles.  Worked Perfectly.<br />No Problems yet.<br />Kind of wish I had gone with one of those multi blade peelers but this was a little cheeper, and I ofern find multi use items tend to break often.
Predicted Sentiment: Positive

Text: My 10 month old baby hates this stuff!!  I've tried giving it to him on 4 separate occasions-but everytime he tastes it--he makes a disgusted look and won't eat anymore.  I even tried mixing it with his favorite foods and he just takes one bite and looses his appetite and won't eat anymore of anything for t